<a href="https://colab.research.google.com/github/Galahexolion/Giliojo-mokymo-sistem-taikymai/blob/main/Lab10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# --- 1. BIBLIOTEKŲ DIEGIMAS ---
# Hugging Face bibliotekos nėra standartiškai įdiegtos, todėl jas reikia įdiegti
# 'transformers': modeliai
# 'datasets': duomenų rinkiniai
# 'evaluate': metrikų skaičiavimas
!pip install transformers datasets evaluate accelerate -q

import os
import numpy as np
import torch
from transformers import pipeline
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate

# Išjungiame 'Weights & Biases' registraciją (kad nereikėtų kurti paskyros lab. darbui)
os.environ["WANDB_DISABLED"] = "true"

# Patikriname, ar turime GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Naudojamas įrenginys: {device}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.3 MB/s eta 0:00:00
Naudojamas įrenginys: cuda


In [2]:
# --- 2. HUGGING FACE PIPELINES (Paprasti pavyzdžiai) ---
# Tai demonstruoja, kaip lengva naudoti paruoštus modelius be apmokymo

print("\n--- 1. Sentiment Analysis Demo ---")
sentiment_pipeline = pipeline("sentiment-analysis", device=0 if device=="cuda" else -1)
data = ["I love deep learning!", "I hate waiting for code to compile."]
results = sentiment_pipeline(data)
print(results)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.



--- 1. Sentiment Analysis Demo ---


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cuda:0


[{'label': 'POSITIVE', 'score': 0.9998645782470703}, {'label': 'NEGATIVE', 'score': 0.9982714653015137}]


In [3]:
# --- 3. FINE-TUNING (Pagrindinė užduotis) ---
# Mes imsime 'DistilBERT' modelį ir apmokysime jį IMDB duomenų rinkinyje

print("\n--- 2. Duomenų paruošimas (IMDB) ---")
# Įkeliame IMDB datasetą
imdb = load_dataset("imdb")

# Dėl greičio imame tik mažą dalį duomenų (pvz., 3000 apmokymui, 300 testavimui)
# Pilnas datasetas (25k) mokytųsi labai ilgai.
small_train_dataset = imdb["train"].shuffle(seed=42).select([i for i in list(range(3000))])
small_test_dataset = imdb["test"].shuffle(seed=42).select([i for i in list(range(300))])

print(f"Mokymo imtis: {len(small_train_dataset)}")
print(f"Testavimo imtis: {len(small_test_dataset)}")

# Tokenizavimas (Teksto vertimas į skaičius)
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_train = small_train_dataset.map(preprocess_function, batched=True)
tokenized_test = small_test_dataset.map(preprocess_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


--- 2. Duomenų paruošimas (IMDB) ---


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [4]:
# --- 4. MODELIO PARUOŠIMAS ---

print("\n--- 3. Modelio įkėlimas ---")
# num_labels=2, nes turime POSITIVE ir NEGATIVE klases
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
model.to(device) # Perkeliame į GPU

# Metrikos (Tikslumas ir F1)
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    acc = accuracy.compute(predictions=predictions, references=labels)
    f1_score = f1.compute(predictions=predictions, references=labels)
    return {"accuracy": acc["accuracy"], "f1": f1_score["f1"]}


--- 3. Modelio įkėlimas ---


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
# --- 5. MOKYMAS (TRAINING) ---

# Nustatom mokymo parametrus (su pataisytu 'eval_strategy')
training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",    # Pataisyta eilutė
    save_strategy="epoch",
    load_best_model_at_end=True,
)

# Sukuriame Trainer objektą (čia ir apibrėžiamas 'trainer')
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

print("\n--- 4. Pradedamas mokymas ---")
trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipython-input-3178283396.py:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



--- 4. Pradedamas mokymas ---


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.337186,0.856667,0.866873
2,No log,0.341271,0.860000,0.866242


TrainOutput(global_step=376, training_loss=0.2876313392152178, metrics={'train_runtime': 276.6437, 'train_samples_per_second': 21.689, 'train_steps_per_second': 1.359, 'total_flos': 782725021021056.0, 'train_loss': 0.2876313392152178, 'epoch': 2.0})

In [11]:
# --- 6. VERTINIMAS (EVALUATION) ---

print("\n--- 5. Galutinis vertinimas ---")
# Dabar 'trainer' jau tikrai egzistuoja
eval_results = trainer.evaluate()

print(f"Testavimo tikslumas (Accuracy): {eval_results['eval_accuracy']:.4f}")
print(f"Testavimo F1 balas: {eval_results['eval_f1']:.4f}")

# Išbandome modelį su savo sakiniu
text = "The movie was absolutely fantastic, I enjoyed every second of it!"
inputs = tokenizer(text, return_tensors="pt").to(device)

# Naudojame modelį prognozei
with torch.no_grad():
    logits = model(**inputs).logits

predicted_class_id = logits.argmax().item()
print(f"\nSakinys: '{text}'")
print(f"Prognozė: {model.config.id2label[predicted_class_id]} (0=NEG, 1=POS)")


--- 5. Galutinis vertinimas ---


Testavimo tikslumas (Accuracy): 0.8567
Testavimo F1 balas: 0.8669

Sakinys: 'The movie was absolutely fantastic, I enjoyed every second of it!'
Prognozė: LABEL_1 (0=NEG, 1=POS)
